<a href="https://colab.research.google.com/github/Asigen93/DeepLearning_Tugas2/blob/main/gorengan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Library

In [13]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, datasets, models
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import cv2

# Mount Google Drive

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load and Transform Data

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Ganti path sesuai lokasi dataset kamu di Google Drive
data_dir = "/content/drive/MyDrive/Deep Learning (Gorengan)"
dataset = datasets.ImageFolder(data_dir, transform=transform)

# Placing Data into a DataFrame

In [12]:
image_paths = [path[0] for path in dataset.imgs]
labels = [path[1] for path in dataset.imgs]
class_names = dataset.classes

df = pd.DataFrame({"Image Path": image_paths, "Label": labels})
df["Label Name"] = df["Label"].apply(lambda x: class_names[x])
print(df.head())

NameError: name 'dataset' is not defined

# Visualizing images from the dataset

In [ ]:
def show_sample_images(dataset, class_names):
    fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    for i, cls in enumerate(class_names):
        idx = dataset.class_to_idx[cls]
        img_path, _ = dataset.imgs[np.where(np.array(labels)==idx)[0][0]]
        image = Image.open(img_path)
        axes[i].imshow(image)
        axes[i].set_title(cls)
        axes[i].axis("off")
    plt.show()

    show_sample_images(dataset, class_names)

# Error Rate Function

In [ ]:
def compute_error_rate(y_true, y_pred):
    return 1 - np.mean(np.array(y_true) == np.array(y_pred))

# Data Preprocessing

In [ ]:
batch_size = 32
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)

# Load EfficientNet-B0

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = models.efficientnet_b0(pretrained=True)
for param in model.features.parameters():
    param.requires_grad = False

model.classifier[1] = nn.Linear(model.classifier[1].in_features, len(class_names))
model = model.to(device)

# Training the model

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

train_losses, val_losses = [], []

for epoch in range(1, 51):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    train_losses.append(running_loss / len(train_loader))

    # Validation
    model.eval()
    val_loss = 0.0
    correct, total = 0, 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_losses.append(val_loss / len(val_loader))
    acc = 100 * correct / total
    print(f"Epoch {epoch}/50, Train Loss: {train_losses[-1]:.4f}, Val Loss: {val_losses[-1]:.4f}, Val Acc: {acc:.2f}%")

# Visualizing Loss Curves

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Validation Loss")
plt.title("Loss Curves")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

# Model Evaluation

In [ ]:
all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

print(classification_report(all_labels, all_preds, target_names=class_names))
cm = confusion_matrix(all_labels, all_preds)

plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt="d", xticklabels=class_names, yticklabels=class_names, cmap="Blues")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

# Making predictions on the Test Data

In [ ]:
def predict_image(image_path, model, transform, class_names):
    image = Image.open(image_path).convert("RGB")
    image_tensor = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image_tensor)
        _, prediction = torch.max(output, 1)
    return class_names[prediction.item()]

print("Prediction:", predict_image(df["Image Path"].iloc[10], model, transform, class_names))

# Grad-CAM Visualization

In [ ]:
def grad_cam(model, image_path, class_idx=None):
    image = Image.open(image_path).convert("RGB")
    input_tensor = transform(image).unsqueeze(0).to(device)

    model.eval()
    features_blobs = []

    def hook_feature(module, input, output):
        features_blobs.append(output)

    handle = model.features[-1].register_forward_hook(hook_feature)

    with torch.no_grad():
        output = model(input_tensor)
        pred_idx = output.argmax(dim=1).item() if class_idx is None else class_idx
        score = output[0, pred_idx]

    model.zero_grad()
    score.backward(retain_graph=True)

    gradients = model.features[-1][0].weight.grad
    activations = features_blobs[0].squeeze(0)

    weights = torch.mean(gradients, dim=[1, 2])
    cam = torch.zeros(activations.shape[1:], dtype=torch.float32)

    for i, w in enumerate(weights):
        cam += w * activations[i]

    cam = torch.clamp(cam, min=0).cpu().numpy()
    cam = cv2.resize(cam, (224, 224))
    cam -= np.min(cam)
    cam /= np.max(cam)

    img = cv2.cvtColor(np.array(image.resize((224,224))), cv2.COLOR_RGB2BGR)
    heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
    superimposed = heatmap * 0.4 + img

    plt.figure(figsize=(10,4))
    plt.subplot(1,2,1)
    plt.imshow(image)
    plt.title("Original Image")
    plt.subplot(1,2,2)
    plt.imshow(superimposed[...,::-1].astype(np.uint8))
    plt.title("Grad-CAM")
    plt.show()

    handle.remove()

# Contoh penggunaan Grad-CAM
grad_cam(model, df["Image Path"].iloc[10])